In [1]:
# Populate data into Milvus vdb

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import pymilvus
import time
from pathlib import Path
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
from sqlalchemy import create_engine, MetaData
from sqlalchemy.engine import URL
import pypyodbc

start_time = time.time()

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## SQL queries slot in here

###
# declares the variables and defines them for the server connections, along with the table names that are going to be assigned
SERVER_NAME = 'tcp:tecafs.database.windows.net,1433'
DATABASE_NAME = 'TecafsSqlDatabase'
TABLE_NAME = 'clean_data'

###################################

# makes the connection to the database with the connection string; has the driver, server name, database name, id, and password
connection_string = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Uid={'tecafs2023'};
    Pwd={'Capstone50'};
"""
###################################

connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})
engine = create_engine(connection_url, module=pypyodbc)

# Define the SQL query

sql_query = """
SELECT *
FROM hist_data
WHERE Time >= '2021-01-01' AND Time < '2023-01-01'
AND Location = 'WEST'
ORDER BY Time
"""

'''
sql_query = """
SELECT *
FROM hist_data
WHERE Time >= '2021-01-01' AND Time < '2021-01-15'
"""
'''

# Execute the query and load data into a DataFrame
df = pd.read_sql(sql_query, engine)

##


#format dataframe:
#drop dew point, wind direction, wind gust, pressure, precip
df = df.drop(columns=['dew point', 'wind', 'wind gust', 'pressure', 'precip.'])
df = df.rename(columns={"wind speed": "wind_speed"})
#keep temperature, humidity, wind speed, condition
#check column names are the same btwn df and vdb
#make separate year, date, hour columns while keeping datetime in time column. Change references to time to use hour instead
df['year'] = df['time'].dt.year.astype(int)
df['date'] = df['time'].dt.strftime('%m-%d')
df['hour'] = df['time'].dt.strftime('%H:%M')

def get_embedding(prevec):
    text = [prevec] #remove brackets should fix extra bracket issue
    embedding = model.encode(text, show_progress_bar = False)
    return embedding

# Create the prevec conglomeration by combining values from other columns and embed it into 'vector' column 
df['vector'] = df.apply(lambda row: get_embedding(f"{row['year']}/{row['date']}/{row['hour']}/{row['hour']}/{row['hour']}/{row['temperature']}/{row['temperature']}/{row['humidity']}/{row['wind_speed']}/{row['condition']}"), axis=1)
# Fix formatting (remove extra brackets)
df['vector'] = df['vector'].apply(lambda row: row[0])

df = df.sort_values(by=['location', 'time']) #sorts by datetime within each location
df = df.reset_index(drop=True)

#reorder columns to match vdb
df = df[['vector', 'location', 'time', 'year', 'date', 'hour', 'temperature', 'humidity', 'wind_speed', 'condition', 'power']]

#convert time to string
df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['temperature'] = df['temperature'].astype(int)
df['location'] = df['location'].astype(str)
df['time'] = df['time'].astype(str)
df['date'] = df['date'].astype(str)
df['hour'] = df['hour'].astype(str)
df['condition'] = df['condition'].astype(str)
print(df)

#write to csv
data_folder = Path("/mnt/c/Users/wcans/milvus_compose/ERCOT_Hourly_Load_2022/")
output_csv_file_path = data_folder / 'west_historical_data.csv'  # Replace with the desired output path
df.to_csv(output_csv_file_path, index=False)

connections.connect(
  alias="default",
  user='root',
  password='tecafs',
  host='localhost',
  port='19530'
)
connections.list_connections()

# Prepare milvus collection to add data
collection_name = "historical_data_21_22"
collection = Collection(collection_name)

# Create an IVF_FLAT index for collection.
index_params = {
    'metric_type':'L2',
    'index_type':"IVF_FLAT",
    'params':{'nlist': 1536}
}
collection.create_index(field_name="vector", index_params=index_params)
collection.load()

# Insert data
collection.insert(df)

collection.load()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Processing Time: {elapsed_time} seconds")

                                                  vector location  \
0      [-0.041150495, -0.00836642, 0.066553004, 0.062...     WEST   
1      [-0.0060177385, 0.0077668186, 0.059657097, 0.0...     WEST   
2      [0.0005267556, 0.0040374836, 0.059729144, 0.07...     WEST   
3      [-0.0027436456, -0.04365136, 0.044879925, 0.05...     WEST   
4      [-0.00021410684, 0.011399332, 0.07054785, 0.07...     WEST   
...                                                  ...      ...   
17247  [-0.04855808, -0.043173064, -0.037474513, -0.0...     WEST   
17248  [-0.032763153, -0.03446123, -0.049594697, -0.0...     WEST   
17249  [-0.029671824, -0.03848643, -0.023526689, -0.0...     WEST   
17250  [-0.025713248, -0.036114927, -0.04812947, -0.0...     WEST   
17251  [-0.03465798, -0.021478895, -0.050492782, -0.0...     WEST   

                      time  year   date   hour  temperature  humidity  \
0      2021-01-01 00:00:00  2021  01-01  00:00           31      96.0   
1      2021-01-01 01:00:0

In [4]:
print(df)

                                                   vector location  \
0       [-0.03719126, -0.03798319, -0.01857576, -0.006...    COAST   
1       [-0.029328812, -0.02539734, 0.0037053353, 0.01...    COAST   
2       [-0.03571963, -0.042456143, -0.00014093575, 0....    COAST   
3       [-0.025943207, -0.022099506, -0.0026181194, 0....    COAST   
4       [0.022386014, -0.031345084, 0.07873036, 0.0673...    COAST   
...                                                   ...      ...   
135810  [-0.04855808, -0.043173064, -0.037474513, -0.0...     WEST   
135811  [-0.032763153, -0.03446123, -0.049594697, -0.0...     WEST   
135812  [-0.029671824, -0.03848643, -0.023526689, -0.0...     WEST   
135813  [-0.025713248, -0.036114927, -0.04812947, -0.0...     WEST   
135814  [-0.03465798, -0.021478895, -0.050492782, -0.0...     WEST   

                       time  year   date   hour  temperature  humidity  \
0       2021-01-15 00:00:00  2021  01-15  00:00           56      42.0   
1       202

In [14]:
data_folder = Path("/mnt/c/Users/wcans/milvus_compose/ERCOT_Hourly_Load_2022/")
output_csv_file_path = data_folder / '21_22_historical_data.csv'  # Replace with the desired output path
#df.to_csv(output_csv_file_path, index=False)

#locationslist = ['COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
locationslist = ['WEST']

for i in range(1):
    insert_df = df[df['location'] == locationslist[i]]
    connections.connect(
      alias="default",
      user='root',
      password='tecafs',
      host='localhost',
      port='19530'
    )
    connections.list_connections()
    
    # Prepare milvus collection to add data
    collection_name = "historical_data_21_22"
    collection = Collection(collection_name)
    
    # Create an IVF_FLAT index for collection.
    '''index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{'nlist': 1536}
    }
    collection.create_index(field_name="vector", index_params=index_params)'''
    collection.load()
    
    # Insert data
    collection.insert(insert_df)
    
    collection.load()
    print('inserted', locationslist[i])


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Processing Time: {elapsed_time} seconds")

inserted WEST
Processing Time: 11758.636568546295 seconds


In [13]:
print(df)

                                                   vector location  \
0       [-0.03719126, -0.03798319, -0.01857576, -0.006...    COAST   
1       [-0.029328812, -0.02539734, 0.0037053353, 0.01...    COAST   
2       [-0.03571963, -0.042456143, -0.00014093575, 0....    COAST   
3       [-0.025943207, -0.022099506, -0.0026181194, 0....    COAST   
4       [0.022386014, -0.031345084, 0.07873036, 0.0673...    COAST   
...                                                   ...      ...   
135810  [-0.04855808, -0.043173064, -0.037474513, -0.0...     WEST   
135811  [-0.032763153, -0.03446123, -0.049594697, -0.0...     WEST   
135812  [-0.029671824, -0.03848643, -0.023526689, -0.0...     WEST   
135813  [-0.025713248, -0.036114927, -0.04812947, -0.0...     WEST   
135814  [-0.03465798, -0.021478895, -0.050492782, -0.0...     WEST   

                       time  year   date   hour  temperature  humidity  \
0       2021-01-15 00:00:00  2021  01-15  00:00           56      42.0   
1       202